In [1]:
# Bitcoin Mainnet - Conectar a nodo semilla, handshake real y propagar mensaje "XD"
# ⚠️ ADVERTENCIA: Esto conecta a la red REAL de Bitcoin

import socket
import struct
import hashlib
import time
import random

# === CONFIGURACIÓN MAINNET ===
MAINNET_MAGIC = 0xD9B4BEF9
MAINNET_SEEDS = [
    "seed.bitcoin.sipa.be",
    "dnsseed.bluematt.me",
    "dnsseed.bitcoin.dashjr.org",
    "seed.bitcoinstats.com",
    "seed.bitcoin.jonasschnelli.ch",
    "seed.btc.petertodd.org",
    "seed.bitcoin.sprovoost.nl",
    "dnsseed.emzy.de"
]
PORT = 8333
PROTOCOL_VERSION = 70016

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def checksum(payload):
    return double_sha256(payload)[:4]

def create_message(command, payload=b''):
    magic = struct.pack('<I', MAINNET_MAGIC)
    cmd = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    chk = checksum(payload)
    return magic + cmd + length + chk + payload

def create_version_payload():
    version = struct.pack('<i', PROTOCOL_VERSION)
    services = struct.pack('<Q', 0)  # No ofrecemos servicios
    timestamp = struct.pack('<q', int(time.time()))

    # Dirección del receptor (dummy)
    addr_recv = struct.pack('<Q', 1)  # services
    addr_recv += b'\x00' * 10 + b'\xff\xff' + socket.inet_aton('0.0.0.0')
    addr_recv += struct.pack('>H', PORT)

    # Nuestra dirección (dummy)
    addr_from = struct.pack('<Q', 0)
    addr_from += b'\x00' * 10 + b'\xff\xff' + socket.inet_aton('0.0.0.0')
    addr_from += struct.pack('>H', 0)

    nonce = struct.pack('<Q', random.randint(0, 2**64-1))
    user_agent = b'\x0F/XD-Messenger:1/'  # Nuestro user agent especial
    start_height = struct.pack('<i', 0)
    relay = struct.pack('<?', False)

    return version + services + timestamp + addr_recv + addr_from + nonce + user_agent + start_height + relay

def parse_message(data):
    if len(data) < 24:
        return None, None, data

    magic = struct.unpack('<I', data[:4])[0]
    if magic != MAINNET_MAGIC:
        return None, None, data

    command = data[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
    payload_len = struct.unpack('<I', data[16:20])[0]

    if len(data) < 24 + payload_len:
        return None, None, data

    payload = data[24:24+payload_len]
    remaining = data[24+payload_len:]

    return command, payload, remaining

def parse_varint(data, offset=0):
    first = data[offset]
    if first < 0xfd:
        return first, offset + 1
    elif first == 0xfd:
        return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
    elif first == 0xfe:
        return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
    else:
        return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9

def parse_addr(payload):
    """Parsear mensaje 'addr' para obtener peers"""
    peers = []
    try:
        count, offset = parse_varint(payload)
        for _ in range(min(count, 10)):  # Limitar a 10 peers
            if offset + 30 > len(payload):
                break
            # timestamp (4) + services (8) + IP (16) + port (2) = 30 bytes
            offset += 4 + 8  # Skip timestamp y services
            ip_bytes = payload[offset:offset+16]
            offset += 16
            port = struct.unpack('>H', payload[offset:offset+2])[0]
            offset += 2

            # Verificar si es IPv4 (mapped en IPv6)
            if ip_bytes[:12] == b'\x00' * 10 + b'\xff\xff':
                ip = socket.inet_ntoa(ip_bytes[12:16])
                if ip not in ['0.0.0.0', '127.0.0.1']:
                    peers.append((ip, port))
    except Exception as e:
        print(f"  ⚠️ Error parseando addr: {e}")
    return peers

def create_alert_payload():
    """Crear un payload de alerta con mensaje XD (formato legacy)"""
    # El mensaje de alerta de Bitcoin (deprecated pero aún procesado por algunos nodos)
    message = b"XD"

    # Estructura del alert payload
    alert_payload = b''
    alert_payload += struct.pack('<i', PROTOCOL_VERSION)  # version
    alert_payload += struct.pack('<q', int(time.time()) - 3600)  # relayUntil
    alert_payload += struct.pack('<q', int(time.time()) + 86400)  # expiration
    alert_payload += struct.pack('<i', random.randint(1, 100000))  # ID
    alert_payload += struct.pack('<i', 0)  # cancel
    alert_payload += b'\x00'  # setCancel (varint 0)
    alert_payload += struct.pack('<i', 0)  # minVer
    alert_payload += struct.pack('<i', 999999)  # maxVer
    alert_payload += b'\x00'  # setSubVer (varint 0)
    alert_payload += struct.pack('<i', 100)  # priority

    # El mensaje "XD"
    alert_payload += bytes([len(message)]) + message  # comment
    alert_payload += bytes([len(message)]) + message  # statusBar
    alert_payload += b'\x00'  # reserved

    # El alert message completo necesita firma (que no tenemos)
    # Pero podemos enviar el payload sin firma válida
    serialized = bytes([len(alert_payload)]) + alert_payload
    signature = b'\x00'  # Firma vacía (los nodos lo rechazarán pero lo procesarán)

    return serialized + signature

def create_inv_payload():
    """Crear inv falso para propagar (los nodos pedirán los datos)"""
    # Tipo 0 = ERROR, 1 = TX, 2 = BLOCK
    # Usamos tipo 1 (TX) con hash que contiene "XD"
    inv_type = struct.pack('<I', 1)  # MSG_TX
    # Hash que representa "XD"
    fake_hash = hashlib.sha256(b"XD - Hola Bitcoin Network!").digest()

    count = b'\x01'  # 1 inventory
    return count + inv_type + fake_hash

def connect_and_handshake(ip, port, timeout=10):
    """Conectar a un peer y hacer handshake completo"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(timeout)

    try:
        print(f"\n🔌 Conectando a {ip}:{port}...")
        sock.connect((ip, port))
        print(f"✅ Conectado!")

        # Enviar VERSION
        version_msg = create_message('version', create_version_payload())
        sock.send(version_msg)
        print("📤 VERSION enviado")

        # Recibir respuestas
        buffer = b''
        got_version = False
        got_verack = False
        peers_found = []

        start = time.time()
        while time.time() - start < timeout:
            try:
                data = sock.recv(4096)
                if not data:
                    break
                buffer += data

                while True:
                    command, payload, buffer = parse_message(buffer)
                    if command is None:
                        break

                    print(f"📥 Recibido: {command} ({len(payload) if payload else 0} bytes)")

                    if command == 'version':
                        got_version = True
                        # Enviar VERACK
                        sock.send(create_message('verack'))
                        print("📤 VERACK enviado")
                        # Pedir direcciones
                        sock.send(create_message('getaddr'))
                        print("📤 GETADDR enviado - Pidiendo peers")

                    elif command == 'verack':
                        got_verack = True
                        print("🤝 HANDSHAKE COMPLETADO!")

                    elif command == 'ping':
                        nonce = payload[:8] if len(payload) >= 8 else b'\x00' * 8
                        sock.send(create_message('pong', nonce))
                        print("📤 PONG enviado")

                    elif command == 'addr':
                        new_peers = parse_addr(payload)
                        peers_found.extend(new_peers)
                        print(f"📋 Recibidos {len(new_peers)} peers")

                    elif command == 'sendheaders':
                        pass  # Ignorar
                    elif command == 'sendcmpct':
                        pass  # Ignorar
                    elif command == 'feefilter':
                        pass  # Ignorar

                if got_version and got_verack and len(peers_found) > 0:
                    break

            except socket.timeout:
                continue

        return sock, got_version and got_verack, peers_found

    except Exception as e:
        print(f"❌ Error: {e}")
        sock.close()
        return None, False, []

def propagate_xd(sock, peers):
    """Propagar mensaje XD a través de la red"""
    print("\n" + "="*50)
    print("🚀 PROPAGANDO MENSAJE 'XD' A LA RED BITCOIN")
    print("="*50)

    # Método 1: Enviar INV con hash "XD"
    print("\n📤 Enviando INV con hash 'XD'...")
    inv_msg = create_message('inv', create_inv_payload())
    sock.send(inv_msg)
    print("✅ INV enviado - Los nodos pedirán esta 'transacción'")

    # Método 2: Intentar enviar a más peers
    propagated_to = 1
    for ip, port in peers[:5]:  # Intentar con hasta 5 peers más
        try:
            print(f"\n🔌 Conectando a peer adicional {ip}:{port}...")
            peer_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            peer_sock.settimeout(5)
            peer_sock.connect((ip, port))

            # Handshake rápido
            peer_sock.send(create_message('version', create_version_payload()))
            time.sleep(1)
            data = peer_sock.recv(4096)
            if b'version' in data:
                peer_sock.send(create_message('verack'))
                time.sleep(0.5)
                # Enviar INV
                peer_sock.send(inv_msg)
                print(f"✅ INV 'XD' enviado a {ip}")
                propagated_to += 1
            peer_sock.close()
        except Exception as e:
            print(f"⚠️ No se pudo conectar a {ip}: {e}")

    return propagated_to

def main():
    print("="*60)
    print("🌐 BITCOIN MAINNET - PROPAGADOR DE MENSAJE 'XD'")
    print("="*60)
    print("\n⚠️ CONECTANDO A LA RED REAL DE BITCOIN (MAINNET)")
    print("   Esto es solo para demostración educativa\n")

    # Paso 1: Resolver DNS seed para obtener nodo inicial
    print("📡 PASO 1: Contactando DNS Seeds...")
    seed_peers = []

    for seed in MAINNET_SEEDS:
        try:
            print(f"  Resolviendo {seed}...")
            ips = socket.gethostbyname_ex(seed)[2]
            seed_peers.extend([(ip, PORT) for ip in ips[:3]])
            print(f"    ✅ {len(ips)} IPs encontradas")
            if len(seed_peers) >= 5:
                break
        except Exception as e:
            print(f"    ❌ Error: {e}")

    if not seed_peers:
        print("❌ No se pudieron resolver DNS seeds")
        return

    print(f"\n📋 {len(seed_peers)} peers candidatos de DNS seeds")

    # Paso 2: Conectar a un peer random y hacer handshake
    print("\n🎲 PASO 2: Seleccionando peer random para handshake...")
    random.shuffle(seed_peers)

    connected_sock = None
    network_peers = []

    for ip, port in seed_peers[:3]:
        sock, success, peers = connect_and_handshake(ip, port)
        if success:
            connected_sock = sock
            network_peers = peers
            print(f"\n✅ Handshake exitoso con {ip}:{port}")
            print(f"📋 Obtuvimos {len(peers)} peers de la red")
            break
        elif sock:
            sock.close()

    if not connected_sock:
        print("❌ No se pudo completar handshake con ningún peer")
        return

    # Paso 3: Propagar mensaje XD
    print("\n📢 PASO 3: Propagando mensaje 'XD' a la red...")

    total_propagated = propagate_xd(connected_sock, network_peers)

    # Esperar un poco para ver respuestas
    print("\n⏳ Esperando respuestas de la red (10 segundos)...")
    connected_sock.settimeout(2)

    responses = 0
    start = time.time()
    while time.time() - start < 10:
        try:
            data = connected_sock.recv(4096)
            if data:
                cmd, payload, _ = parse_message(data)
                if cmd:
                    print(f"📥 Respuesta: {cmd}")
                    if cmd == 'getdata':
                        responses += 1
                        print("   🎯 ¡Un nodo quiere el mensaje 'XD'!")
        except:
            pass

    connected_sock.close()

    # Resumen final
    print("\n" + "="*60)
    print("📊 RESUMEN DE LA OPERACIÓN")
    print("="*60)
    print(f"  ✅ Handshake completado: SÍ (MAINNET REAL)")
    print(f"  📤 Mensaje 'XD' propagado a: {total_propagated} nodos")
    print(f"  📥 Nodos que respondieron: {responses}")
    print(f"  🌐 Peers descubiertos: {len(network_peers)}")
    print("\n  ℹ️ El INV con hash 'XD' fue enviado a la red.")
    print("     Los nodos enviarán GETDATA pidiendo la 'transacción'")
    print("     pero como no existe, eventualmente lo ignorarán.")
    print("="*60)

# Ejecutar
main()

🌐 BITCOIN MAINNET - PROPAGADOR DE MENSAJE 'XD'

⚠️ CONECTANDO A LA RED REAL DE BITCOIN (MAINNET)
   Esto es solo para demostración educativa

📡 PASO 1: Contactando DNS Seeds...
  Resolviendo seed.bitcoin.sipa.be...
    ✅ 25 IPs encontradas
  Resolviendo dnsseed.bluematt.me...
    ✅ 21 IPs encontradas

📋 6 peers candidatos de DNS seeds

🎲 PASO 2: Seleccionando peer random para handshake...

🔌 Conectando a 134.101.142.46:8333...
✅ Conectado!
📤 VERSION enviado
📥 Recibido: version (102 bytes)
📤 VERACK enviado
📤 GETADDR enviado - Pidiendo peers
📥 Recibido: wtxidrelay (0 bytes)
📥 Recibido: sendaddrv2 (0 bytes)
📥 Recibido: verack (0 bytes)
🤝 HANDSHAKE COMPLETADO!
📥 Recibido: sendcmpct (9 bytes)
📥 Recibido: ping (8 bytes)
📤 PONG enviado
📥 Recibido: feefilter (8 bytes)
📥 Recibido: addr (30003 bytes)
📋 Recibidos 7 peers

✅ Handshake exitoso con 134.101.142.46:8333
📋 Obtuvimos 7 peers de la red

📢 PASO 3: Propagando mensaje 'XD' a la red...

🚀 PROPAGANDO MENSAJE 'XD' A LA RED BITCOIN

📤 Enviando 